#Classifying AG News Dataset
### Shailesh Patro

While doing this assignment, I used the tutorial presented here: https://machinelearningmastery.com/use-word-embedding-layers-deep-learning-keras/. The model presented in this tutorial uses word embeddings which is superior to the bag of words model.

In [25]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [0]:
import pandas as pd
import keras
import numpy as np

from sklearn.model_selection import train_test_split
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Conv1D, BatchNormalization, Activation,Flatten
from keras.layers import Embedding, Input, Dense, Dropout, Lambda, MaxPooling1D
from keras.optimizers import SGD
from keras.models import Model


dataPath = '/content/gdrive/My Drive/Colab Notebooks/ag_news_csv/'

In [0]:
train = pd.read_csv(dataPath+'train.csv', names=["label", "title", "text"])
test = pd.read_csv(dataPath+'test.csv', names=["label","title", "text"])

In [35]:
x_train = train["title"] + " " + train["text"]
y_train = train["label"]

x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.1, random_state=123)
x_train
# y_train

54257     Pakistanis ban religious meetings Authorities ...
57940     Marks  amp; Spencer Says First-Half Earnings F...
19952     SFA DELAY MOLDOVA DECISION The Scottish Footba...
9864      Deutsche Bank hit again by phishing attack Aft...
116957    Top British Official Resigns Amid Scandal A ca...
8612      Japan to Deport Ex-Chess Champion Bobby Fische...
48105     Moats sinks Fresno St. RUSTON, La. -- Ryan Moa...
13701     40 injured in French motorway pileup Bordeaux,...
29994     Afghan President Escapes Attack Afghanistan Pr...
38872     Roddick Powers U.S. to Lead in Davis Semis CHA...
62115     Walsh in charge for Masconomet Myles Walsh sto...
92763     Wholesale prices jump in October by 1.7 percen...
3707      Sprint stars create new Greek tragedy ATHENS, ...
10502     Server sales rocket by almost a quarter HP and...
52501     Ethics Panel Rebukes DeLay Twice in a Week WAS...
106083    Barrera Proved he #39;s Better than Morales Th...
98416     Alien aims to multiply and thr

In [0]:
x_test = test["title"] + " " + test["text"]
y_test = test["label"]


In [37]:
num_class = 4
vocab = 50000 # size of vocabulary
max_length = 1024

encode_train = [one_hot(d,vocab) for d in x_train] #[one_hot(d,vocab) for d in x_train]
encode_val = [one_hot(d,vocab) for d in x_val]
encode_test = [one_hot(d,vocab) for d in x_test]
pad_train = np.array(pad_sequences(encode_train, maxlen=max_length, padding='post'))
pad_val = np.array(pad_sequences(encode_val, maxlen=max_length, padding='post'))
pad_test = np.array(pad_sequences(encode_test, maxlen=max_length, padding='post'))
trainLen = pad_train.shape[0]
valLen = pad_val.shape[0]
testLen = pad_test.shape[0]
pad_train = pad_train.flatten()
pad_val = pad_val.flatten()
pad_test = pad_test.flatten()

pad_train = pad_train.reshape(trainLen,max_length)
pad_val = pad_val.reshape(valLen,max_length)
pad_test = pad_test.reshape(testLen,max_length)

y_train[y_train==4] = 0
y_val[y_val==4] = 0
y_test[y_test==4] = 0
y_train = keras.utils.to_categorical(y_train, num_class)
y_val = keras.utils.to_categorical(y_val, num_class)
y_test = keras.utils.to_categorical(y_test, num_class)


/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [38]:
model = Sequential()
model.add(Embedding(vocab, 512, input_length=max_length))
model.add(Flatten())

model.add(Dense(4, activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
print(model.summary())
model.fit(pad_train,y_train, epochs=4, batch_size=1024,
    validation_data=(pad_val,y_val), verbose=0)

scores = model.evaluate(pad_test,y_test)
print('Test loss: ', scores[0])
print('Test accuracy:',scores[1])


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 1024, 512)         25600000  
_________________________________________________________________
flatten_2 (Flatten)          (None, 524288)            0         
_________________________________________________________________
dense_2 (Dense)              (None, 4)                 2097156   
Total params: 27,697,156
Trainable params: 27,697,156
Non-trainable params: 0
_________________________________________________________________
None
7600/7600 [==============================] - 11s 1ms/step
('Test loss: ', 0.2649510206438993)
('Test accuracy:', 0.9131578947368421)
